In [1]:
import os
#os.chdir(r"C:\Users\Aditya\GitHub\zavmo\zavmo-api\zavmo")
# os.chdir("/Users/adityachhabra/Github/zavmo/zavmo-api/zavmo")

In [2]:
os.chdir(f"{os.path.dirname(os.getcwd())}/zavmo")

In [3]:
os.getcwd()

'/Users/mumtaz/Documents/projects/zavmo/zavmo-api/zavmo'

In [4]:
from dotenv import load_dotenv
load_dotenv()
from IPython.display import Markdown

In [30]:
from helpers.chat import get_prompt
import yaml
import codecs


def get_yaml_data(yaml_path, yaml_dir="assets/data"):
    """Load a YAML file containing field data.

    Args:
        yaml_path (str): Path to the YAML file.
        yaml_dir (str, optional): Directory containing the YAML file. Defaults to 'assets/data'.

    Returns:
        dict: A dictionary of field data.

    Raises:
        UnicodeDecodeError: If there's an encoding issue with the file.
        yaml.YAMLError: If there's an issue parsing the YAML content.
        FileNotFoundError: If the specified file doesn't exist.
    """
    # Check if yaml_path ends with .yaml or .yml
    if not yaml_path.endswith(('.yaml', '.yml')):
        yaml_path += '.yaml'
    yaml_path = os.path.join(yaml_dir, yaml_path)
    
    try:
        with codecs.open(yaml_path, 'r', encoding='utf-8') as f:
            return yaml.safe_load(f)
    except UnicodeDecodeError as e:
        print(f"Encoding error in file {yaml_path}: {e}")
        print("Make sure the file is saved with UTF-8 encoding.")
        raise
    except yaml.YAMLError as e:
        print(f"YAML parsing error in file {yaml_path}: {e}")
        raise
    except FileNotFoundError:
        print(f"File not found: {yaml_path}")
        raise

def get_agent_instructions(stage_name: str) -> str:
    """
    Compile instructions for the agent.
    
    Args:
        stage_name (str): The name of the current stage.

    Returns:
        str: Instructions for the agent.
    """
    conf_data       = get_yaml_data(stage_name.lower())
    agent_keys      = ['name', 'description', 'instructions', 'examples', 'completion_condition', 'next_stage', 'next_stage_description']
    prompt_context  = {k:v for k,v in conf_data.items() if k in agent_keys}

    system_content  = get_prompt('probe.md', 
                                    context=prompt_context,
                                    prompt_dir="assets/prompts")
    return system_content   

In [5]:
from helpers.swarm import fetch_agent_response, run_step

In [6]:
from pydantic import BaseModel, Field
from typing import List, Dict, Literal, Optional
from helpers._types import (
    Agent,
    StrictTool,
    Result,
)

In [18]:

class Module(BaseModel):
    title: str = Field(description="The title of the module. It can be an Assessment Area or a learner's interest area")
    learning_outcomes: List[str] = Field(description="List upto 5 learning outcomes.")
    lessons: List[str] = Field(description="List of lessons in this module based on the learning outcomes")
    duration: int = Field(description="The total duration of the module in hours")

class Curriculum(StrictTool):
    """Generate a detailed curriculum for the learner based on the NOS Assessment Areas and OFQUAL standards."""
    title: str = Field(description="The title of the curriculum")
    subject: str = Field(description="The main subject area of the curriculum based on the NOS Assessment Areas.")
    level: str = Field(description="The difficulty level of the curriculum (e.g., beginner, intermediate, advanced)")
    prerequisites: List[str] = Field(description="Any prerequisites needed to undertake this curriculum")
    modules: List[Module] = Field(description="List upto 7 modules mainly designed on NOS Assessment Areas shared.")
        
    def execute(self, context: Dict):
        return Result(value=str(self.model_dump()), context=context)    


In [21]:
instructions = get_agent_instructions('discuss')

In [22]:
# Markdown(instructions)

In [23]:
discuss_agent = Agent(
    name="Discussion",
    id="discuss",
    instructions=instructions,
    functions=[
        Curriculum
    ],
    parallel_tool_calls=False,
    tool_choice='auto',
    model="gpt-4o"
)


In [24]:
import json
message_history = json.loads(open("/Users/mumtaz/Documents/projects/zavmo/zavmo-api/docs/test_history.json").read())

In [25]:
message_history[-1]['content'] ="""Discussion data updated successfully for mumtaz@example.com
        
**Timeline**: 9
**Learning Style**: Case Study Discussions

**Data for Curriculum Generation**:

**NOS Assessment Area**: Understanding of product details

**OFQUAL Standards for the NOS Assessment Area**: 
['Understand how a marketing strategy supports the sales function', 'Analyse how a marketing strategy supports an organisation', 'Evaluate the relationships between marketing, sales and other functions', 'Analyse how a marketing strategy supports the sales function', 'Understand marketing research', 'Evaluate the use of up to date and relevant market information to support sales-related decisions', 'Differentiate between primary and secondary data', 'Identify and interpret trends in sales-related data', 'Differentiate between sources of market information and evaluate the validity of data', 'Evaluate a range of analytical tools to analyse sales-related information and why each one might be chosen in a given situation', 'Analyse the suitability, relevance, validity and reliability of market information', 'Differentiate between ways of presenting quantitative and qualitative market information', 'Understand the use of marketing research in sales', 'Review how to obtain feedback from colleagues about the usefulness of sales-related information', 'Evaluate budget allocations for obtaining market information', 'Evaluate the importance of the sales function contributing realistic market information to the marketing function', 'Evaluate how an organisation’s policy on the storage of information can meet data protection requirements', 'Appraise how the use of sales related information can result in changes to the marketing mix and the marketing and sales strategy', 'Be able to use market analyses in sales', 'Employ market analyses to inform the development of a sales strategy that takes account of market trends and competitor activity', 'Review market analysis to identify a clear positioning message that aims to satisfy target', 'Review the competitive edge of organisation’s products/services', 'Complete the financial return to the organisation resulting from the targeting of customers', 'Be able to progress sales through alignment with marketing strategies', 'Identify how the organisation can benefit from addressing the requirements of target markets', 'Review how new products/services allow the organisation to satisfy customers’ changing needs and how this generates return on investment', 'Appraise how marketing strategies and plans help to develop and maintain positive relationships with the organisation’s sales team(s) and customers, and the resulting benefits in terms of customer loyalty and competitive advantage', 'Evaluate the value of marketing in progressing sales']

**NOS Assessment Area**: Controlling customer conversations

**OFQUAL Standards for the NOS Assessment Area**: 
['Understand the principles of developing strategic relationships with major customers', 'Be able to identify major customers and develop major customer plans that are mutually beneficial', 'Be able to develop strategic relationships with major customers to meet mutual objectives', 'Be able to evaluate the success of strategic relationship activities and plan for future activities', 'Explain the principles of developing mutually beneficial relationships with major customers ', 'Evaluate distinctions between transactional selling and major customer management', 'Evaluate advantages and dangers of major customer management to own organisation and customer organisations', 'Develop criteria to use to select major customers based on strategic long term value of customers', 'Analyse methods to obtain buy-in from senior management and colleagues for major customer management', 'Explain the stages of major customer relationship development', 'Evaluate behaviours which build up a relationship of trust with a major customer', 'Identify the customer(s) which fulfil the organisation’s criteria to be classed as a major customer', 'Analyse the major customer’s business and their corporate business and marketing strategy', 'Analyse the current and future strategic and operational business challenges faced by the major customers', 'Identify and agree prioritised common business objectives between own organisation and the major customer, which are consistent with both organisations’ financial objectives and business strategies', 'Assess with the major customer strategic options towards achieving the common business objectives which involve mutual working', 'Work in partnership with the major customer to develop a plan and actions towards addressing the identified challenges and common objectives', 'Analyse expertise required in own team to work with the major customers', 'Co-ordinate business functions across own organisation to develop strategic relationships with the major customers', 'Evaluate opportunities for creating long term value for the major customer', 'Evaluate methods for reducing risk to the major customer regarding own organisation’s products and services', 'Monitor and critically evaluate the outcomes of activities in developing the strategic relationship with the major customer, including the return on the investment being made', 'Develop a plan to address any issues regarding the outcome of strategic relationship activities', 'Develop a succession plan for maintaining a successful relationship with the major customer, with contingencies should particular individuals no longer be able to work with the major customer', 'Evaluate systems for storing and planning major customer information']

**NOS Assessment Area**: Observing signals during customer interactions

**OFQUAL Standards for the NOS Assessment Area**: 
['Understand how to identify customer groups', 'Describe methods of identifying customer groups', 'Compare the characteristics of customers in identified customer groups', 'Understand the motivations, attitudes, and behaviours of customers in identified customer groups', 'Describe the motivations, attitudes, and behaviours of customers in identified customer groups', 'Describe clusters of customers who have distinct similarities in their motivations, attitudes, and behaviours', 'Understand how to profile existing customer groups', 'Identify the characteristics of established customers', 'Describe the similarities between identified established customers', 'Explain how to develop profiles for customer groups']
        
"""

In [26]:
print(message_history[-1]['content'])

Discussion data updated successfully for mumtaz@example.com
        
**Timeline**: 9
**Learning Style**: Case Study Discussions

**Data for Curriculum Generation**:

**NOS Assessment Area**: Understanding of product details

**OFQUAL Standards for the NOS Assessment Area**: 
['Understand how a marketing strategy supports the sales function', 'Analyse how a marketing strategy supports an organisation', 'Evaluate the relationships between marketing, sales and other functions', 'Analyse how a marketing strategy supports the sales function', 'Understand marketing research', 'Evaluate the use of up to date and relevant market information to support sales-related decisions', 'Differentiate between primary and secondary data', 'Identify and interpret trends in sales-related data', 'Differentiate between sources of market information and evaluate the validity of data', 'Evaluate a range of analytical tools to analyse sales-related information and why each one might be chosen in a given situati

In [27]:
response = run_step(
            agent=discuss_agent,
            messages=message_history,
            context={},
            max_turns=5
        )

2025-01-10 16:52:09,268 - INFO - Running step 1 with agent Discussion
2025-01-10 16:52:09,269 - INFO - Logging from fetch_agent response
2025-01-10 16:52:17,484 - INFO - Executing tool: Curriculum with args: {'title': 'Advanced Sales Strategies: Integration with CRM and Negotiation', 'subject': 'Sales and Business Development', 'level': 'Advanced', 'prerequisites': ['Intermediate knowledge of sales techniques', 'Basic understanding of CRM tools'], 'modules': [{'title': 'Product Knowledge and Marketing Integration', 'learning_outcomes': ['Enhance understanding of product features', "Understand marketing's role in sales success", 'Align sales strategies with customer needs'], 'lessons': ['Deep Dive into Product Features', 'The Role of Marketing in Sales', 'Aligning Strategies with Customer Needs'], 'duration': 12}, {'title': 'Negotiation Techniques Mastery', 'learning_outcomes': ['Master negotiation tactics', 'Resolve customer objections effectively', 'Close sales strategically'], 'lesso

In [29]:
# Parse the response to get the curriculum arguments
curriculum_args = response.messages[0]['tool_calls'][0]['function']['arguments']
curriculum_data = json.loads(curriculum_args)
curriculum_data

{'title': 'Advanced Sales Strategies: Integration with CRM and Negotiation',
 'subject': 'Sales and Business Development',
 'level': 'Advanced',
 'prerequisites': ['Intermediate knowledge of sales techniques',
  'Basic understanding of CRM tools'],
 'modules': [{'title': 'Product Knowledge and Marketing Integration',
   'learning_outcomes': ['Enhance understanding of product features',
    "Understand marketing's role in sales success",
    'Align sales strategies with customer needs'],
   'lessons': ['Deep Dive into Product Features',
    'The Role of Marketing in Sales',
    'Aligning Strategies with Customer Needs'],
   'duration': 12},
  {'title': 'Negotiation Techniques Mastery',
   'learning_outcomes': ['Master negotiation tactics',
    'Resolve customer objections effectively',
    'Close sales strategically'],
   'lessons': ['Advanced Negotiation Tactics',
    'Handling Objections Gracefully',
    'Strategic Closing Techniques'],
   'duration': 10},
  {'title': 'Customer Relati

In [59]:
import inspect

In [18]:
from django.core.cache import cache

In [ ]:
active_agent   = discuss_agent
messages       = []
context        = {"email":"aditya@gmail.com","sequence_id":'52442',
                  "stage_data":{'profile':{},'discuss':{},'delivery':{},'demonstrate':{}},
                  "stage":1}

In [ ]:
result = run_step(active_agent, messages, context=context, max_turns=5)

messages.extend(result.messages)
context.update(result.context)
if result.agent!=active_agent:
    active_agent = result.agent
    context['stage'] += 1
    print(f"Switching agent to {active_agent.name}")
print(context)

Markdown(messages[-1]['content'])

In [ ]:
user_resp = "Sounds good, let's go"

messages.append({"role":"user","content":user_resp})